# Imports and Setup

## Installing the `SciScript` Library

In [1]:

!git clone https://github.com/sciserver/SciScript-Python.git/

Cloning into 'SciScript-Python'...
remote: Enumerating objects: 1354, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 1354 (delta 104), reused 164 (delta 87), pack-reused 1161
Receiving objects: 100% (1354/1354), 423.95 KiB | 10.34 MiB/s, done.
Resolving deltas: 100% (769/769), done.


In [2]:
cd SciScript-Python/py3

/kaggle/working/SciScript-Python/py3


In [3]:
!python setup.py install

running install
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
/opt/conda/lib/python3.7/site-packages/setuptools/command/easy_install.py:159: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  EasyInstallDeprecationWarning,
running bdist_egg
running egg_info
creating SciServer.egg-info
writing SciServer.egg-info/PKG-INFO
writing dependency_links to SciServer.egg-info/dependency_links.txt
writing top-level names to SciServer.egg-info/top_level.txt
writing manifest file 'SciServer.egg-info/SOURCES.txt'
writing manifest file 'SciServer.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/SciServer
copying SciServer/Jobs.py -> bui

## Imports

In [1]:
from SciServer import CasJobs  # Communicate between SciServer Compute and CasJobs
from SciServer import Authentication  # Authenticate users
# from kaggle_secrets import UserSecretsClient #Kaggle secrets
import pandas as pd
import numpy as np
import datetime
import os

## Authenticating

In [5]:
token = "6ee0c47a31754ae1bef3d7e701f6fb3e"
Authentication.setToken(token)

Let's see if the authentication worked.

In [ ]:
query = """select top 10 objid, ra, dec, r
from galaxy
where clean = 1"""

df = CasJobs.executeQuery(sql=query, context="DR18")
df

So, it did work! Now the main tasks.

# Loading Data

## Some Functions

Here, some functions can be found to help with the data download.

In [8]:
def tables_format(tables):
    """Formats the table dictionary returned by `CasJobs.getTables()` to be more human-readable
    
    Parameters
    ----------
    tables : dict
        Dictionary of table information returned by CasJobs
    
    Returns
    ---------
    None
    """
    tables = sorted(tables, key=lambda k: k["Name"])  # alphabetize by table name

    tables_final = pd.DataFrame()
    names = []
    rows = []
    sizes = []
    createds = []

    for table in tables:
        names.append(table["Name"])
        rows.append(table["Rows"])
        sizes.append(table["Size"])

        create_date = table["Date"]
        create_sec = (
            create_date / 10000000
        ) 
        firstday = datetime.datetime(1, 1, 1, 0, 0)  # Save 1 AD as "firstday"
        created = firstday + datetime.timedelta(
            seconds=create_sec
        )
        createds.append(created.strftime("%Y-%m-%d %H:%M:%S"))
    tables_final["Name"] = names
    tables_final["Rows"] = rows
    tables_final["Size (KB)"] = sizes
    tables_final["Created"] = createds
    return tables_final

def job_describer(job_description):
    """Prints the results of the CasJobs job status functions in a human-readable manner

    Parameters
    ----------
    job_description : dict
        Dictionary of job status information returned by CasJobs
    
    Returns
    -------
    None
    """

    code_to_status = {
        0: "Not Started",
        1: "Started",
        5: "Completed",
        2: "Cancelling",
        3: "Cancelled",
        4: "Failed",
    }
    status = job_description["Status"]
    if status in code_to_status:
        status = code_to_status[status]
    else:
        status = "Unknown"
    
    if job_description["Status"] == 0:
        print("The Job has not started yet.")
        return 
    
    job_ID = job_description["JobID"]
    target = job_description["Target"]
    message = job_description["Message"]
    create_table = job_description["Created_Table"]
    rows = job_description["Rows"]
    start_time = pd.to_datetime(job_description["TimeStart"])
    end_time = pd.to_datetime(job_description["TimeEnd"])
    submit_time = pd.to_datetime(job_description["TimeSubmit"])

    wait_time = (start_time - submit_time).total_seconds()
    run_time = (end_time - start_time).total_seconds()

    print(f"Job ID: {job_ID}")
    print(f"Status: {status}")
    print(f"Target: {target}")
    print(f"Created Table: {create_table}")
    print(f"Rows: {rows}")
    print(f"Message: {message}")
    print(f"Submit Time: {submit_time}")
    print(f"Start Time: {start_time}")
    print(f"End Time: {end_time}")
    print(f"Wait Time: {wait_time} seconds")
    print(f"Run Time: {run_time} seconds")

## What Data to Include?

First, start with lower columns and rows. If performance is satisfactory, load more columns and/or rows.

>{c} = u, g, r, i, z

Columns which we are going to use:
`PhotoPrimary` View
- objID, run, camcol, field (For identification)
- rowv, colv (all deg/day)
- modelFlux_{c}, {c}, psfMag_{c} (mag or nanomaggies)
- petroRad_{c}, deVRad_{c}, expRad_{c} (arcsec)
- q_{c}, u_{c}, expAB_{c} (Some ratios)
- ra, dec, b, l (for position)
- type

Conditions which we are going to use:
- nChild = 0
- clean = 1

```sql
  (sa.specObjID=0) and
  (sa.psfmag_u < 35) and
  (sa.psfmag_g < 35) and
  (sa.psfmag_r < 35) and
  (sa.psfmag_i < 35) and
  (sa.psfmag_z < 35) and
  (sa.psfmag_u > 0) and
  (sa.psfmag_g > 0) and
  (sa.psfmag_r > 0) and
  (sa.psfmag_i > 0) and
  (sa.psfmag_z > 0) and
  ```

Let's write the query:

```sql
SELECT objID, run, camcol, field, type,
rowv, colv,
psfMag_u, psfMag_g, psfMag_r, psfMag_i, psfMag_z,
petroRad_u, petroRad_g, petroRad_r, petroRad_i, petroRad_z, 
q_u, q_g, q_r, q_i, q_z,
ra, dec, b, l
FROM PhotoPrimary
WHERE
  (nchild = 0) and
  (clean = 1) and
  (specObjID=0) and
  (psfmag_u < 35) and
  (psfmag_g < 35) and
  (psfmag_r < 35) and
  (psfmag_i < 35) and
  (psfmag_z < 35) and
  (psfmag_u > 0) and
  (psfmag_g > 0) and
  (psfmag_r > 0) and
  (psfmag_i > 0) and
  (psfmag_z > 0) and
```

Let's see how many columns there are:

In [8]:
query = """SELECT type, COUNT(*) as num_samples
into mydb.info
FROM PhotoPrimary
WHERE
  (clean = 1) and
  (specObjID = 0) and
  (psfmag_u < 35) and
  (psfmag_g < 35) and
  (psfmag_r < 35) and
  (psfmag_i < 35) and
  (psfmag_z < 35) and
  (psfmag_u > 0) and
  (psfmag_g > 0) and
  (psfmag_r > 0) and
  (psfmag_i > 0) and
  (psfmag_z > 0)
GROUP BY type"""

print(query)
# job_id = CasJobs.submitJob(sql=query, context="DR18")
# job_description = CasJobs.waitForJob(jobId=job_id)
# job_describer(job_description=job_description)

SELECT type, COUNT(*) as num_samples
into mydb.info
FROM PhotoPrimary
WHERE
  (clean = 1) and
  (specObjID = 0) and
  (psfmag_u < 35) and
  (psfmag_g < 35) and
  (psfmag_r < 35) and
  (psfmag_i < 35) and
  (psfmag_z < 35) and
  (psfmag_u > 0) and
  (psfmag_g > 0) and
  (psfmag_r > 0) and
  (psfmag_i > 0) and
  (psfmag_z > 0)
GROUP BY type


I've commented the above job submission as I don't want to run it again. The result is saved in `info` table.

In [9]:
q = """SELECT *
FROM info
"""
data = CasJobs.executeQuery(sql=q, context="mydb")
data.head()

,type,num_samples
0,3,131561928
1,6,174567056


We can see that there are over 300 Million samples. We can not hope to load all the data that easily. In fact, we will be working with just a few couple Million of samples (Say, 3 million for type STAR and 3 million for type GALAXY?).

CasJobs provides 500,000 kB of storage for MyDB database. This will set the limit on how much data can we store at once. So, we will write queries which does not exceed this memory limit. Then we will download the table as gz file. Some part of this process is going to be manual.

Here is the final query
```sql
SELECT objID, run, camcol, field, type, -- identification
rowv, colv, -- velocity
u, g, r, i, z, -- modelMag
psfMag_u, psfMag_g, psfMag_r, psfMag_i, psfMag_z, -- psfMag
modelFlux_u, modelFlux_g, modelFlux_r, modelFlux_i, modelFlux_z, -- modelFlux
petroRad_u, petroRad_g, petroRad_r, petroRad_i, petroRad_z, -- petroRad
expRad_u, expRad_g, expRad_r, expRad_i, expRad_z, -- expRad
q_u, q_g, q_r, q_i, q_z, -- q ratio
u_u, u_g, u_r, u_i, u_z, -- u ratio
expAB_u, expAB_g, expAB_r, expAB_i, expAB_z, -- exp a/b ratio
ra, dec, b, l -- position
INTO MyDB.data_1
FROM PhotoPrimary
WHERE
  (nchild = 0) and
  (clean = 1) and
  (type = 3) and -- galaxy
  (specObjID=0) and
  (psfmag_u < 35) and
  (psfmag_g < 35) and
  (psfmag_r < 35) and
  (psfmag_i < 35) and
  (psfmag_z < 35) and
  (psfmag_u > 0) and
  (psfmag_g > 0) and
  (psfmag_r > 0) and
  (psfmag_i > 0) and
  (psfmag_z > 0)

--order by OBJID
ORDER BY objID

--paging
OFFSET 0 ROWS
FETCH NEXT 100000 ROWS ONLY
```


The offset and the next rows will be changed. Also, `type = 3` (which correspond to galaxy) will be changed to `type = 6` for stars.

Here is the parameterized query.

In [15]:
i = 4

length = 1000000
offset = 2000000
type_ = 3
table_index = i+1

query = f"""SELECT objID, run, camcol, field, type, -- identification
rowv, colv, -- velocity
u, g, r, i, z, -- modelMag
psfMag_u, psfMag_g, psfMag_r, psfMag_i, psfMag_z, -- psfMag
modelFlux_u, modelFlux_g, modelFlux_r, modelFlux_i, modelFlux_z, -- modelFlux
petroRad_u, petroRad_g, petroRad_r, petroRad_i, petroRad_z, -- petroRad
expRad_u, expRad_g, expRad_r, expRad_i, expRad_z, -- expRad
q_u, q_g, q_r, q_i, q_z, -- q ratio
u_u, u_g, u_r, u_i, u_z, -- u ratio
expAB_u, expAB_g, expAB_r, expAB_i, expAB_z, -- exp a/b ratio
ra, dec, b, l -- position
INTO MyDB.data_{table_index}
FROM PhotoPrimary
WHERE
  (nchild = 0) and
  (clean = 1) and
  (type = {type_}) and -- 3 galaxy, 6 star
  (specObjID=0) and
  (psfmag_u < 35) and
  (psfmag_g < 35) and
  (psfmag_r < 35) and
  (psfmag_i < 35) and
  (psfmag_z < 35) and
  (psfmag_u > 0) and
  (psfmag_g > 0) and
  (psfmag_r > 0) and
  (psfmag_i > 0) and
  (psfmag_z > 0)

--order by OBJID
ORDER BY objID

--paging
OFFSET {offset} ROWS
FETCH NEXT {length} ROWS ONLY"""

print(query)

SELECT objID, run, camcol, field, type, -- identification
rowv, colv, -- velocity
u, g, r, i, z, -- modelMag
psfMag_u, psfMag_g, psfMag_r, psfMag_i, psfMag_z, -- psfMag
modelFlux_u, modelFlux_g, modelFlux_r, modelFlux_i, modelFlux_z, -- modelFlux
petroRad_u, petroRad_g, petroRad_r, petroRad_i, petroRad_z, -- petroRad
expRad_u, expRad_g, expRad_r, expRad_i, expRad_z, -- expRad
q_u, q_g, q_r, q_i, q_z, -- q ratio
u_u, u_g, u_r, u_i, u_z, -- u ratio
expAB_u, expAB_g, expAB_r, expAB_i, expAB_z, -- exp a/b ratio
ra, dec, b, l -- position
INTO MyDB.data_5
FROM PhotoPrimary
WHERE
  (nchild = 0) and
  (clean = 1) and
  (type = 3) and -- 3 galaxy, 6 star
  (specObjID=0) and
  (psfmag_u < 35) and
  (psfmag_g < 35) and
  (psfmag_r < 35) and
  (psfmag_i < 35) and
  (psfmag_z < 35) and
  (psfmag_u > 0) and
  (psfmag_g > 0) and
  (psfmag_r > 0) and
  (psfmag_i > 0) and
  (psfmag_z > 0)

--order by OBJID
ORDER BY objID

--paging
OFFSET 2000000 ROWS
FETCH NEXT 1000000 ROWS ONLY


We will run just one job in this notebook. Othersjobs can similarly be run. The next notebook will be used to download and load data.

In [16]:
job_id = CasJobs.submitJob(sql=query, context="DR18")

In [20]:
job_description= CasJobs.getJobStatus(job_id)
job_description

{'JobID': 60011351,
 'Rows': 1000000,
 'WebServicesID': 1676402349,
 'TimeSubmit': '2023-04-02T07:26:55.25',
 'TimeStart': '2023-04-02T07:27:00.597',
 'TimeEnd': '2023-04-02T07:29:20.833',
 'SendEmail': 0,
 'Status': 5,
 'AutoComplete': 0,
 'Estimate': 500,
 'TaskName': 'SciScript-Python.CasJobs.submitJob',
 'OutputLoc': '',
 'HostIP': 'DR18_long      ',
 'Message': 'Query Complete',
 'Query': 'SELECT objID, run, camcol, field, type, -- identification\nrowv, colv, -- velocity\nu, g, r, i, z, -- modelMag\npsfMag_u, psfMag_g, psfMag_r, psfMag_i, psfMag_z, -- psfMag\nmodelFlux_u, modelFlux_g, modelFlux_r, modelFlux_i, modelFlux_z, -- modelFlux\npetroRad_u, petroRad_g, petroRad_r, petroRad_i, petroRad_z, -- petroRad\nexpRad_u, expRad_g, expRad_r, expRad_i, expRad_z, -- expRad\nq_u, q_g, q_r, q_i, q_z, -- q ratio\nu_u, u_g, u_r, u_i, u_z, -- u ratio\nexpAB_u, expAB_g, expAB_r, expAB_i, expAB_z, -- exp a/b ratio\nra, dec, b, l -- position\nINTO MyDB.data_5\nFROM PhotoPrimary\nWHERE\n  (nchil

In [21]:
job_description = CasJobs.waitForJob(jobId=job_id)
job_describer(job_description=job_description)

Job ID: 60011351
Status: Completed
Target: DR18
Created Table: data_5
Rows: 1000000
Message: Query Complete
Submit Time: 2023-04-02 07:26:55.250000
Start Time: 2023-04-02 07:27:00.597000
End Time: 2023-04-02 07:29:20.833000
Wait Time: 5.3469999999999995 seconds
Run Time: 140.236 seconds


Links:

1. http://skyservice.pha.jhu.edu/CasJobsOutput/CSV/data_1_hari31416.csv.gz
2. http://skyservice.pha.jhu.edu/CasJobsOutput/CSV/data_2_hari31416.csv.gz
3. http://skyservice.pha.jhu.edu/CasJobsOutput/CSV/data_3_hari31416.csv.gz
4. http://skyservice.pha.jhu.edu/CasJobsOutput/CSV/data_4_hari31416.csv.gz
5. http://skyservice.pha.jhu.edu/CasJobsOutput/CSV/data_5_hari31416.csv.gz

In [22]:
q = """DROP TABLE IF EXISTS data_1
DROP TABLE IF EXISTS data_2
DROP TABLE IF EXISTS data_3
DROP TABLE IF EXISTS data_4
DROP TABLE IF EXISTS data_5"""
CasJobs.executeQuery(sql=q, context="MyDB")

,Rows Affected
0,0
